### IMPORTING NECESSARY PACKAGES

In [1]:
import requests
import pandas as pd 

### TASK 1 : CLASS ScriptData, fetching data from Alpha Vantage API

In [2]:
class ScriptData:
    
    #constructor
    def __init__(self, key):
        self.key = key
        self.base_url = "https://www.alphavantage.co/query?"  #base url api key and endpoint attached by user
        self.data ={}                                         #dictionary for storing the data from API
        
     
    
    #overridden function __getitem__()
    def __getitem__(self, script):
        if script in self.data:
            return self.data[script]
        else:
            return None
        
        
    #overridden function __setitem__() 
    def __setitem__(self, key, value):
        self.data[key] = value
        
    #overridden function __contains__() 
    def __contains__(self, key):
        return key in self.data
    
    
    #task 1.a fetching data from the api
    #accepts the argument as company name
    def fetch_intraday_data(self,script) :
        if script in self.data :                     #if already fetched no need to refetch
            return self.data[script]                 #its totally controversial since in real case the data may get updated
        
        parameters = {                               # api endpoints along with user's key
            "function" : "TIME_SERIES_INTRADAY",
            "symbol" : script,
            "interval" : "1min",
            "apikey" : self.key
        }
        response = requests.get(self.base_url, params=parameters)
        
        if response.status_code == 200:                #sever code 200 => OK 
            self.data[script]=response.json()          #else API down or client errors (not relevant to us for now)
            return self.data[script]
        else:
            return None

    #task 1.b converting data to a dataframe using pandas library 
    #accepts the argument as company name
    def convert_intraday_data(self,script) :
        data = self.fetch_intraday_data(script)        #fetching data from member method
        if data is None:
            return None
        
        del data["Meta Data"]                          #cleaning
        timestamp_data=list(list())                    #list of list helper 2d array
        for key in data['Time Series (1min)'] :
            temp=[]
            temp.append(key)
            for k in data['Time Series (1min)'][key].values() :
                temp.append(k)
            timestamp_data.append(temp)
    
        columns = ['timestamp','open','high','low','close','volume']
        df = pd.DataFrame(timestamp_data,columns=columns)
        
        del self.data[script]                          #overwriting the dictionary "data"
        self.data[script]=df                         

### Data Display for GOOGL and AAPL

In [3]:
key = "RCLRXKEBTO17UXET"                               #user specific key 
script_data = ScriptData(key)
script_data.fetch_intraday_data("GOOGL")
script_data.convert_intraday_data("GOOGL")
script_data["GOOGL"]

,timestamp,open,high,low,close,volume
0,2023-02-27 20:00:00,90.0800,90.0800,89.9900,89.9900,1109
1,2023-02-27 19:59:00,90.0000,90.0000,90.0000,90.0000,337
2,2023-02-27 19:56:00,90.0000,90.0000,90.0000,90.0000,540
3,2023-02-27 19:55:00,90.0900,90.0900,90.0900,90.0900,198
4,2023-02-27 19:54:00,90.0000,90.0000,90.0000,90.0000,415
...,...,...,...,...,...,...
95,2023-02-27 16:00:00,89.8450,89.9100,89.8400,89.8800,437950
96,2023-02-27 15:59:00,89.7900,89.8600,89.7850,89.8500,201995
97,2023-02-27 15:58:00,89.7750,89.8200,89.7650,89.8000,222436
98,2023-02-27 15:57:00,89.7400,89.8400,89.7300,89.7750,190726


In [4]:
script_data.fetch_intraday_data("AAPL")
script_data.convert_intraday_data("AAPL")
script_data["AAPL"]

,timestamp,open,high,low,close,volume
0,2023-02-27 20:00:00,148.0200,148.0300,148.0200,148.0300,745
1,2023-02-27 19:59:00,148.0100,148.0300,148.0100,148.0300,1308
2,2023-02-27 19:58:00,148.0100,148.0200,148.0100,148.0200,2252
3,2023-02-27 19:57:00,148.0000,148.0000,148.0000,148.0000,336
4,2023-02-27 19:56:00,148.0000,148.0000,148.0000,148.0000,501
...,...,...,...,...,...,...
95,2023-02-27 16:12:00,147.9900,148.0100,147.9200,148.0000,22606
96,2023-02-27 16:11:00,148.0000,148.0000,147.9800,147.9800,1174
97,2023-02-27 16:10:00,147.9200,147.9200,147.9200,147.9200,64063
98,2023-02-27 16:09:00,147.9900,148.0000,147.9200,147.9900,94171


In [5]:
'GOOGL' in script_data

True

In [6]:
'AAPL' in script_data

True

In [7]:
'NVDA' in script_data

False

### TASK 2 : Indicator1 Function with dataFrame timestamp and indicator

In [8]:
# task 2 implementation of indicator1 function for finding mean with "timeperiod" in assignment given as 5
def indicator1(df,timeperiod) :
    columns = ['timestamp','close']
    col_rename=['timestamp','indicator']
    
    new_df = df[columns].rename(columns = 
                                {
                                   'close': 'indicator',
                                }
                               ).copy()
    
    new_df['indicator']=new_df['indicator'].rolling(window=timeperiod).mean()
    return new_df

In [9]:
indicator1(script_data["GOOGL"],5)

,timestamp,indicator
0,2023-02-27 20:00:00,NaN
1,2023-02-27 19:59:00,NaN
2,2023-02-27 19:56:00,NaN
3,2023-02-27 19:55:00,NaN
4,2023-02-27 19:54:00,90.01600
...,...,...
95,2023-02-27 16:00:00,89.88802
96,2023-02-27 15:59:00,89.88200
97,2023-02-27 15:58:00,89.87400
98,2023-02-27 15:57:00,89.84300


In [10]:
indicator1(script_data["AAPL"],5)

,timestamp,indicator
0,2023-02-27 20:00:00,NaN
1,2023-02-27 19:59:00,NaN
2,2023-02-27 19:58:00,NaN
3,2023-02-27 19:57:00,NaN
4,2023-02-27 19:56:00,148.016
...,...,...
95,2023-02-27 16:12:00,148.004
96,2023-02-27 16:11:00,148.002
97,2023-02-27 16:10:00,147.982
98,2023-02-27 16:09:00,147.976


### TASK 3 : CLASS Strategy inheriting ScriptData and performing some operations 

In [11]:
class Strategy(ScriptData) :
    #constructor
    def __init__(self,key) :
        super().__init__(key)                                     #inheriting from base class
        
        
    #task 3 creating a dataframe named signals for displaying the signals to BUY or SELL stocks    
    def getSignals(self,val,timeperiod) :
        df = self.data[val]
        df1 = indicator1(df,timeperiod)
        
        
        df['close'] = df['close'].astype(float)                   #data was initially in float 
        diff_indicator = df1['indicator'].diff()               
        diff_close = df['close'].diff()
        buy_sig = diff_indicator > 0                              #boolean flags 
        sell_sig = diff_indicator < 0                            
        no_sig = (diff_indicator == 0) | (diff_close == 0)

        signals = pd.DataFrame({'timestamp': df['timestamp'],'signal': 'NO_SIGNAL'}) #instantiating a dataframe
        signals.loc[buy_sig & (diff_close < 0), 'signal'] = 'BUY'
        signals.loc[sell_sig & (diff_close > 0), 'signal'] = 'SELL'
        val = "NO_SIGNAL"
        signals = signals.drop(index=signals[signals['signal'] == val].index)         #dropping the rows with no signals
        return signals

In [12]:
strategy = Strategy("GOOGL")
strategy.fetch_intraday_data("GOOGL")
strategy.convert_intraday_data("GOOGL")
strategy['GOOGL']
strategy.getSignals('GOOGL',5)                                    #signals for GOOGL

,timestamp,signal
7,2023-02-27 19:45:00,BUY
8,2023-02-27 19:44:00,SELL
11,2023-02-27 19:33:00,SELL
13,2023-02-27 19:25:00,BUY
14,2023-02-27 19:21:00,BUY
17,2023-02-27 19:11:00,SELL
21,2023-02-27 18:51:00,BUY
25,2023-02-27 18:32:00,SELL
26,2023-02-27 18:31:00,SELL
29,2023-02-27 18:11:00,BUY


In [13]:
strategy = Strategy("AAPL")
strategy.fetch_intraday_data("AAPL")
strategy.convert_intraday_data("AAPL")
strategy['AAPL']
strategy.getSignals('AAPL',5)                                                #signals for AAPl

,timestamp,signal
8,2023-02-27 19:47:00,BUY
12,2023-02-27 19:43:00,SELL
26,2023-02-27 19:10:00,BUY
27,2023-02-27 19:07:00,BUY
28,2023-02-27 19:01:00,BUY
33,2023-02-27 18:45:00,BUY
35,2023-02-27 18:37:00,BUY
40,2023-02-27 18:22:00,BUY
44,2023-02-27 18:16:00,SELL
47,2023-02-27 18:04:00,BUY
